In [20]:
import mne 
from glob import glob 
import os
import matplotlib
from openpyxl import load_workbook
import numpy as np

In [21]:
matplotlib.use('Qt5Agg') 

In [22]:
#quanti soggetti ho nella mia cartella?
subs_file_path=glob("..\..\DATASET PRINCIPALE\CartellaEEG/*.vhdr")
print(len(subs_file_path))

2


In [23]:
#creo vettore file excel---- !ATTENZIONE FAR RUNNARE IL FILE EXCEL.YPTR per aggiustare gli excel
excel_file_path=glob("..\..\DATASET PRINCIPALE\Cartellaexcel/*.xlsx")
print(len(excel_file_path))

2


In [24]:
#Creo la funzione che legge e processa i dati 
def prep_data(file_path):
    data= mne.io.read_raw_brainvision(file_path, preload=True)
    data.set_eeg_reference();
    data.filter(l_freq=0.1,h_freq=100)
    data.resample(250)
    return data

#Creo la funzione che estrae la matrice delle annotazioni dai file excel 
def get_vectors(data_path):

    book=load_workbook(data_path)
    sheet=book.active
    
    #onset
    onset=[]
    for row in sheet['a2':'a22249']:
        for cel in row:
               onset.append(cel.value)
                
    onset=np.array(onset)
    onset=onset*0.001
    
    #duration 
    duration=np.ones((22248,), dtype=int)*0.01
    
    #description 
    animal=['animal']
    description=[]
    for row in sheet ['e2':'e22249']:
        for cel in row:
            if(cel.value in animal):
                description.append(cel.value)
            else: 
                description.append('E 1')
    return [onset, duration, description] 

#creo la funzione che applica le annotazioni 
def fix_events(data_vector, matrix):
    myannot=[mne.Annotations(onset=matrix[0][0], duration=matrix[0][1],description=matrix[0][2])for i in range(len(excel_file_path))]
    for i in range(len(excel_file_path)):
        data_vector[i].set_annotations(myannot[i])
    
    return data_vector

In [25]:
#funzione preprocessing
prep_data_sub_vector= [prep_data(i)for i in subs_file_path]


Extracting parameters from ..\..\DATASET PRINCIPALE\CartellaEEG\sub-01_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3035739  =      0.000 ...  3035.739 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 33001 samples (33.001 sec)

Extracting parameters from ..\..\DATASET PRINCIPALE\CartellaEEG\sub-02_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0

In [26]:
#estraggo la matrice delle annotazioni 
matrix=[]
for i in range(len(excel_file_path)):
    matrix=[get_vectors(i)for i in excel_file_path]

In [27]:
data_vector_ok= fix_events(prep_data_sub_vector, matrix)

In [28]:
events=mne.events_from_annotations(data_vector_ok[0])
events
#prima colonna --> sample number 
#terxa colonna--> event number 

#QUANTI SONO
b=len(events[0][events[0][:,2]==10002])
a=len(events[0][events[0][:,2]==10001])



print("Numero di altro:",a)
print("Numero di animali:",b)

e=len(events[0])
print("Numero totale di eventi:",e)

Used Annotations descriptions: ['E 1', 'animal']
Numero di altro: 20952
Numero di animali: 1296
Numero totale di eventi: 22248


In [29]:
mne.viz.plot_events(events[0])

<Figure size 1280x960 with 1 Axes>